# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases




# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1825  100  1825    0     0  11050      0 --:--:-- --:--:-- --:--:-- 11060
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  39892      0 --:--:-- --:--:-- --:--:-- 39748
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  53820      0 --:--:-- --:--:-- --:--:-- 53942
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  24904      0 --:--:-- --:--:-- --:--:-- 25076
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [4]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [5]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [6]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

Definition
QuantumDrive (Q-1) is a solid-state drive (SSD) implementing the NVMe 2.0 protocol over a PCIe Gen 5 interface. It is positioned for creative professionals, including video editors, 3D artists, and photographers. Available capacities are 2TB, 4TB, and 8TB, and the product carries a 5-year limited warranty.

Function/Operation
- Interface and performance: NVMe 2.0 on PCIe Gen 5 with sequential read speeds up to 7,500 MB/s and sequential write speeds up to 7,000 MB/s. Full performance requires a host platform that supports NVMe/PCIe Gen 5.
- Thermal management: An integrated graphene heat spreader is designed to prevent thermal throttling under sustained load.
- Security and software: Hardware AES-256 encryption is supported. The drive includes the DataWeaver backup and encryption software suite.
- Intended purpose: Reduce workflow bottlenecks and speed rendering and loading in professional workstation environments.

Key Findings/Impact
- Performance and endurance: The 4TB model is rated for 3,000 TBW endurance. Random I/O metrics, latency measurements, controller details, and endurance figures for the 2TB and 8TB models are not specified.
- Workflow impact: Targeted for professional video editing, 3D content creation, and photography, with an emphasis on reducing render and load times. The bundled DataWeaver suite and hardware encryption support secure storage and backup practices.
- Compatibility considerations: Interface support is specified (NVMe 2.0, PCIe Gen 5), but OS/platform compatibility details are not provided.
- Limitations and evidence: Claims of “ending the wait” and preventing thermal throttling are based on product messaging; no third-party benchmarks are provided. Pricing information is not available.
- Availability: Pre-orders are being promoted via the product page.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 115.79 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': '- Definition: QuantumDrive '
                                       '(QuantumDrive Q-1) is a solid-state '
                                       'drive (SSD) using NVMe 2.0 over PCIe '
                                       'Gen 5, positioned for creative '
                                       'professionals such as video editors, '
                                       '3D artists, and photographers.\n'
                                       '- Purpose: Reduce workflow bottlenecks '
                                       'and speed up rendering and loading, '
                                       'framed as “End the Wait” and “Built '
                                       'for Pros.”\n'
                                       '- Core technology:\n'
                                       '  - NVMe

In [7]:
#@title Product Marketing Copy Generation(Use Case 1)
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
Chrono SSD Pro is ChronoTech’s flagship PCIe Gen4 solid-state drive designed for digital artists and content creators. The lineup includes a 4TB model and is available for purchase immediately.

Function/Operation
- Prioritizes raw storage throughput to eliminate workflow bottlenecks.
- Delivers sequential read speeds up to 7,300 MB/s on the PCIe Gen4 platform.
- Engineered to maximize read/write performance so storage does not impede creative tasks.

Key Findings/Impact
- Stated benefits include reduced wait times and accelerated creative workflows.
- Differentiators include a creator-focused, performance-first design and a “highly competitive” price point for the 4TB model.
- The company positions the drive to ensure tools keep pace with creators’ speed and ambition, emphasizing that technology should not hinder creativity.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 177.43 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Explain the mechanism or '
                                  'findings clearly and concisely.", '
                                  '"style_guide": "Maintain an objective and '
                                  'formal tone. Use precise terminology. '
                                  'Prioritize factual accuracy and clarity '
                                  'over narrative flair.", "structure": '
                                  '["Definition", "Function/Operation", "Key '
                                  'Findings/Impact"], "instruction": "Organize '
                                  'the provided facts into the defined '
                                  'structure, adhering to the style_guide."}'},
    'planned_input': { 'intent_query': 'Executive brief blueprint for '
               

In [8]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
Project QuantumDrive (QuantumDrive Q-1) is a PCIe Gen 5 NVMe 2.0 solid-state drive designed for creative professionals, offered in 2TB, 4TB, and 8TB capacities. It features integrated hardware encryption and bundled backup software.

Function/Operation
- Interface: NVMe 2.0 over PCIe Gen 5.
- Performance: Sequential read up to 7,500 MB/s; sequential write up to 7,000 MB/s.
- Thermals: Integrated graphene heat spreader engineered to prevent thermal throttling under sustained workloads.
- Endurance and warranty: 3,000 TBW for the 4TB model; 5-year limited warranty.
- Security and software: AES-256 hardware encryption with the included DataWeaver backup and encryption suite.
- Physical details: Dimensions, weight, and specific form factor are not specified beyond NVMe 2.0 over PCIe Gen 5.

Key Findings/Impact
- Creative workflow acceleration: High sequential throughput is suited to large media file transfers, faster project load times, and efficient handling of video, 3D, and photographic assets.
- Sustained performance: The graphene heat spreader is designed to maintain throughput during prolonged tasks (e.g., renders, exports) by preventing thermal throttling.
- Data protection: Hardware-level AES-256 encryption and the DataWeaver suite support secure storage and backup practices.
- Reliability signals: A 3,000 TBW rating for the 4TB model and a 5-year limited warranty indicate durability expectations for intensive use.
- Scope notes: Endurance figures for capacities other than 4TB and detailed physical specifications are not provided.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 74.60 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Explain the mechanism or '
                                  'findings clearly and concisely.", '
                                  '"style_guide": "Maintain an objective and '
                                  'formal tone. Use precise terminology. '
                                  'Prioritize factual accuracy and clarity '
                                  'over narrative flair.", "structure": '
                                  '["Definition", "Function/Operation", "Key '
                                  'Findings/Impact"], "instruction": "Organize '
                                  'the provided facts into the defined '
                                  'structure, adhering to the style_guide."}'},
    'planned_input': { 'intent_query': 'Blueprint for a short marketing '
              

In [9]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


TL;DR: It’s a clear voice-and-tone playbook with SEO and a 3‑email nurture example. Great guardrails; no hard numbers or case studies.

What’s in:
- Core principles: Clarity (simple, scannable), Confidence (direct, no hedging), Aspiration (future-focused benefits).
- Tone: Be the expert guide—empowering, forward-looking, accessible.
- Guardrails: No unprofessional slang, no universal quantitative promises, no competitor bashing.
- Channel examples:
  - SEO: Pillar page plus supporting posts (e.g., “Ultimate Guide to Video Editing Storage”), authoritative but accessible, link to the QuantumDrive product page where it fits.
  - Email nurture (Problem → Solution → Proof): 
    - 1: Acknowledge slow-storage pain, no product yet.
    - 2: Introduce QuantumDrive with clear benefits and “End the Wait” positioning + CTA.
    - 3: Social proof and 5‑year warranty + pre‑order CTA.
- Consistency cues: Benefit-first, forward-looking, active voice, short sentences, scannable structure across all assets.

What it supports (qualitatively):
- Trust via Confidence.
- Easier customer experience via Clarity.
- Note: No measured outcomes.

What’s not provided:
- Stats on revenue, recognition, conversion, CX, ops (onboarding, revisions, velocity, alignment).
- Case studies or benchmarks.

Bottom line: Use the principles everywhere; if you need proof/metrics, they’re not in these docs.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 97.40 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Persuasive one-page pitch advocating '
                                       'for adopting and using a Brand Tone & '
                                       'Voice Guide. Structure: magnetic hook, '
                                       'problem/pain points from inconsistent '
                                